In [1]:
%run params.ipynb

In [2]:
if 'rules' in PARAMS:
    boolean_rules = PARAMS['rules']
else:
    import rules.example
    boolean_rules = rules.example.build_rules()['rules']

# boolean_rules

In [3]:
%matplotlib inline
import matplotlib
import matplotlib.pylab as pl
import boolean2 as bl
import pandas as pd
import numpy as np
from time import gmtime, strftime
import utils
import warnings
import os

In [4]:
def simulate_chunk(c):
    chunk = []
    model = bl.Model(boolean_rules, mode='sync')
    for i in range(PARAMS["chunk_size"]):
        model.initialize(missing=bl.util.randbool)
        model.iterate(steps=30)

        dic = {}
        (cycle_start, cycle_len) = model.detect_cycles()
        dic['cycle_start'] = cycle_start
        dic['cycle_len'] = cycle_len
        for key, value in sorted(model.data.items()):
            dic['init_' + key] = int(value[0])
            dic['cycle_' + key] = ''.join(
                map(lambda x: str(x * 1), value[cycle_start:(
                    cycle_start + cycle_len)]))
        dic['data'] = model.data
        chunk.append(dic)

    print("Chunk created {}".format(c))
    return chunk    

In [5]:
import multiprocessing
cpu_count = multiprocessing.cpu_count()

print("Running boolean simulations on {} CPUs".format(cpu_count))
pool = multiprocessing.Pool(cpu_count)
chunks = pool.map(simulate_chunk, range(PARAMS["chunks"]))
pool.close()
pool.terminate()

Running boolean simulations on 16 CPUs
Chunk created 4
Chunk created 2
Chunk created 1
Chunk created 3
Chunk created 8
Chunk created 9
Chunk created 7
Chunk created 0
Chunk created 5
Chunk created 6


In [15]:
filename = build_filename(PARAMS, "simulations.h5")
if PARAMS['rewrite'] and os.path.exists(filename):
    print("Removing previous simulations")
    os.remove(filename)

print("Saving chunks to data file")
frame = pd.DataFrame()
for i, chunk in enumerate(chunks):
     frame = frame.append(pd.DataFrame(chunk))

warnings.filterwarnings('ignore')
frame.to_hdf(filename, key="%s" % strftime("chunk_%Y_%m_%d__%H_%M_%S", gmtime()))
warnings.filterwarnings('default')

Saving chunks to data file


In [ ]:
del frame
del chunks